## Building a GPT (MLX version)

Companion notebook to the [Zero To Hero](https://karpathy.ai/zero-to-hero.html) video on GPT.

In [ ]:
# We always start with a dataset to train on. Let's download the tiny shakespeare dataset
!wget -N https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2025-10-28 23:07:52--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  4.59MB/s    in 0.2s    

Last-modified header missing -- time-stamps turned off.
2025-10-28 23:07:53 (4.59 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [ ]:
# read it in to inspect it
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [ ]:
print("length of dataset in characters: ", len(text))

length of dataset in characters:  1115394


In [ ]:
# let's look at the first 1000 characters
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [ ]:
# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [ ]:
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

print(encode("hii there"))
print(decode(encode("hii there")))

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


In [ ]:
# let's now encode the entire text dataset and store it into a MLX array
import mlx.core
data = mlx.core.array(encode(text), dtype=mlx.core.int64)
print(data.shape, data.dtype)
print(data[:1000]) # the 1000 characters we looked at earier will to the GPT look like this

(1115394,) mlx.core.int64
array([18, 47, 56, ..., 8, 0, 0], dtype=int64)


In [ ]:
# Let's now split up the data into train and validation sets
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

In [ ]:
block_size = 8
train_data[:block_size+1]

array([18, 47, 56, ..., 15, 47, 58], dtype=int64)

In [ ]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target: {target}")

when input is array([18], dtype=int64) the target: 47
when input is array([18, 47], dtype=int64) the target: 56
when input is array([18, 47, 56], dtype=int64) the target: 57
when input is array([18, 47, 56, 57], dtype=int64) the target: 58
when input is array([18, 47, 56, 57, 58], dtype=int64) the target: 1
when input is array([18, 47, 56, 57, 58, 1], dtype=int64) the target: 15
when input is array([18, 47, 56, ..., 58, 1, 15], dtype=int64) the target: 47
when input is array([18, 47, 56, ..., 1, 15, 47], dtype=int64) the target: 58


In [ ]:
mlx.core.random.seed(1337)
batch_size = 4 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = mlx.core.random.randint(0, len(data) - block_size, (batch_size,)).tolist()
    x = mlx.core.stack([data[i:i+block_size] for i in ix])
    y = mlx.core.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")

inputs:
(4, 8)
array([[47, 45, 52, ..., 47, 52, 1],
       [57, 0, 16, ..., 1, 57, 43],
       [40, 56, 53, ..., 11, 1, 40],
       [1, 46, 39, ..., 1, 61, 39]], dtype=int64)
targets:
(4, 8)
array([[45, 52, 57, ..., 52, 1, 45],
       [0, 16, 47, ..., 57, 43, 43],
       [56, 53, 49, ..., 1, 40, 43],
       [46, 39, 60, ..., 61, 39, 56]], dtype=int64)
----
when input is [47] the target: 45
when input is [47, 45] the target: 52
when input is [47, 45, 52] the target: 57
when input is [47, 45, 52, 57] the target: 1
when input is [47, 45, 52, 57, 1] the target: 47
when input is [47, 45, 52, 57, 1, 47] the target: 52
when input is [47, 45, 52, 57, 1, 47, 52] the target: 1
when input is [47, 45, 52, 57, 1, 47, 52, 1] the target: 45
when input is [57] the target: 0
when input is [57, 0] the target: 16
when input is [57, 0, 16] the target: 47
when input is [57, 0, 16, 47] the target: 42
when input is [57, 0, 16, 47, 42] the target: 1
when input is [57, 0, 16, 47, 42, 1] the target: 57
when inp

In [ ]:
print(xb) # our input to the transformer

array([[47, 45, 52, ..., 47, 52, 1],
       [57, 0, 16, ..., 1, 57, 43],
       [40, 56, 53, ..., 11, 1, 40],
       [1, 46, 39, ..., 1, 61, 39]], dtype=int64)


In [ ]:
import mlx.core
import mlx.nn

mlx.core.random.seed(1337)

class BigramLanguageModel(mlx.nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = mlx.nn.Embedding(vocab_size, vocab_size)

    def __call__(self, idx, targets=None):

        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (B,T,C)

        if targets is None:
            loss = None
        else:
            # B: batch, T: timestep, C: class
            B, T, C = logits.shape
            logits = logits.reshape(B*T, C)
            targets = targets.reshape(B*T)
            loss = mlx.nn.losses.cross_entropy(logits, targets, reduction='mean')

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # sample from the distribution. pass unnormalized logits, dont softmax
            idx_next = mlx.core.random.categorical(logits, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = mlx.core.concatenate((idx, idx_next), axis=1) # (B, T+1)
            mlx.core.eval(idx) # force eval to release intermediate allocations
        return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

print(decode(m.generate(idx = mlx.core.zeros((1, 1), dtype=mlx.core.int64), max_new_tokens=100)[0].tolist()))


(32, 65)
array(4.15891, dtype=float32)

D.P.e'wn,CZsvq gP-f$fvW3aypokkuSEz?Paw:YCj?M;x
rctpxMvdJMlTZrmCZhXRjYRJUfTgldWbqlwXxclCHIWuAFYEBTwJr


In [ ]:
import mlx.optimizers

# create an optimizer
optimizer = mlx.optimizers.AdamW(learning_rate=1e-3)

In [ ]:
batch_size = 32

# Loss wrapper for value_and_grad
def loss_fn(module, idx, targets):
    _, loss = module(idx, targets)
    return loss

loss_and_grad_fn = mlx.nn.value_and_grad(m, loss_fn)

for steps in range(1000): # increase number of steps for good results...

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    loss, grads = loss_and_grad_fn(m, xb, yb)
    optimizer.update(m, grads)

    mlx.core.eval(m.parameters(), optimizer.state)

print(loss.item())


3.747654914855957


In [ ]:
print(decode(m.generate(idx = mlx.core.zeros((1, 1), dtype=mlx.core.int64), max_new_tokens=500)[0].tolist()))


rO?!$CNDsXCdail:c!'law'uOPGCJJDBsSf,E.XVunoK-rJP-ky oHsxxwu,Ni3UJgZbBMO;s:
oPALGSTE-heWO,tcI3$VaeVYfJsTPdoqT-ebedAWy
Ev:WFmCykXrvetkGbw-3-N!'oW
n
qi:FgOyU3XdQwNr iVItNvRo,JbtDAvcfHSKDkh.caNKrf CMr IGs?lbiNDbgJg'cHB:rRwAu;q&UDPhOdnmc:EjUCZBuG?mF.An-r,EMDfCHfITHs ztXPL U3iSE-dAsTxeq:??i
OUQfArTnZ.HgbaJ!zOFIH-vN sioObAaQqt&3bcREcyIgo:fX3'o
DUDMwzc'K;,Z'u&&zdZVpvm Bsyar$-MunMiJyaOqbyaIMPWriG:glSqdHidw3PhLpPoN.;xUNutuQ&rXjSVaENm;vCqwWviUqy:eHRUYb$RintaB;.fI$gw, hcxJri?tLvylbo&'L;;-LnmjmbJk&icTtybIChQ


## The mathematical trick in self-attention

In [ ]:
# toy example illustrating how matrix multiplication can be used for a "weighted aggregation"
mlx.core.random.seed(42)
a = mlx.core.tril(mlx.core.ones((3, 3), dtype=mlx.core.float32))
a = a / mlx.core.sum(a, 1, keepdims=True)
b = mlx.core.random.randint(0,10,(3,2)).astype(mlx.core.float32)
c = a @ b
print('a=')
print(a)
print('--')
print('b=')
print(b)
print('--')
print('c=')
print(c)

a=
array([[1, 0, 0],
       [0.5, 0.5, 0],
       [0.333333, 0.333333, 0.333333]], dtype=float32)
--
b=
array([[2, 8],
       [5, 2],
       [8, 8]], dtype=float32)
--
c=
array([[2, 8],
       [3.5, 5],
       [5, 6]], dtype=float32)


In [ ]:
# consider the following toy example:

mlx.core.random.seed(1337)
B,T,C = 4,8,2 # batch, time, channels
x = mlx.core.random.normal((B,T,C))
x.shape

(4, 8, 2)

In [ ]:
# We want x[b,t] = mean_{i<=t} x[b,i]
xbow = mlx.core.zeros((B,T,C), dtype=mlx.core.float32)
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1] # (t,C)
        xbow[b,t] = mlx.core.mean(xprev, 0)


In [ ]:
# version 2: using matrix multiply for a weighted aggregation
wei = mlx.core.tril(mlx.core.ones((T, T), dtype=mlx.core.float32))
wei = wei / wei.sum(1, keepdims=True)
xbow2 = wei @ x # (B, T, T) @ (B, T, C) ----> (B, T, C)
mlx.core.allclose(xbow, xbow2)

array(True, dtype=bool)

In [ ]:
# version 3: use Softmax
tril = mlx.core.tril(mlx.core.ones((T, T), dtype=mlx.core.float32))
wei = mlx.core.zeros((T,T), dtype=mlx.core.float32)
wei = mlx.core.where((tril == 0), mlx.core.array(float('-inf')), wei)
wei = mlx.nn.softmax(wei, axis=-1)
xbow3 = wei @ x
mlx.core.allclose(xbow, xbow3)


array(True, dtype=bool)

In [ ]:
# version 4: self-attention!
mlx.core.random.seed(1337)
B,T,C = 4,8,32 # batch, time, channels
x = mlx.core.random.normal((B,T,C))

# let's see a single Head perform self-attention
head_size = 16
key = mlx.nn.Linear(C, head_size, bias=False)
query = mlx.nn.Linear(C, head_size, bias=False)
value = mlx.nn.Linear(C, head_size, bias=False)
k = key(x)   # (B, T, 16)
q = query(x) # (B, T, 16)
wei =  q @ k.transpose((0, 2, 1)) # (B, T, 16) @ (B, 16, T) ---> (B, T, T)

tril = mlx.core.tril(mlx.core.ones((T, T), dtype=mlx.core.float32))
#wei = mlx.core.zeros((T,T), dtype=mlx.core.float32)
wei = mlx.core.where((tril == 0), mlx.core.array(float('-inf')), wei)
wei = mlx.nn.softmax(wei, axis=-1)

v = value(x)
out = wei @ v
#out = wei @ x

out.shape

(4, 8, 16)

In [ ]:
wei[0]

array([[1, 0, 0, ..., 0, 0, 0],
       [0.856269, 0.143731, 0, ..., 0, 0, 0],
       [0.431043, 0.00779574, 0.561161, ..., 0, 0, 0],
       ...,
       [0.219786, 0.260875, 0.325269, ..., 0.00784806, 0, 0],
       [0.390585, 0.0546818, 0.0550628, ..., 0.0648744, 0.143865, 0],
       [0.00486556, 0.0906292, 0.136023, ..., 0.183027, 0.0706174, 0.0119941]], dtype=float32)

Notes:
- Attention is a **communication mechanism**. Can be seen as nodes in a directed graph looking at each other and aggregating information with a weighted sum from all nodes that point to them, with data-dependent weights.
- There is no notion of space. Attention simply acts over a set of vectors. This is why we need to positionally encode tokens.
- Each example across batch dimension is of course processed completely independently and never "talk" to each other
- In an "encoder" attention block just delete the single line that does masking with `tril`, allowing all tokens to communicate. This block here is called a "decoder" attention block because it has triangular masking, and is usually used in autoregressive settings, like language modeling.
- "self-attention" just means that the keys and values are produced from the same source as queries. In "cross-attention", the queries still get produced from x, but the keys and values come from some other, external source (e.g. an encoder module)
- "Scaled" attention additional divides `wei` by 1/sqrt(head_size). This makes it so when input Q,K are unit variance, wei will be unit variance too and Softmax will stay diffuse and not saturate too much. Illustration below

In [ ]:
k = mlx.core.random.normal((B,T,head_size))
q = mlx.core.random.normal((B,T,head_size))
wei = q @ k.transpose((0, 2, 1)) * head_size**-0.5

In [ ]:
k.var()

array(0.924469, dtype=float32)

In [ ]:
q.var()

array(1.0613, dtype=float32)

In [ ]:
wei.var()

array(0.968667, dtype=float32)

In [ ]:
mlx.nn.softmax(mlx.core.array([0.1, -0.2, 0.3, -0.2, 0.5]), axis=-1)

array([0.192498, 0.142606, 0.235117, 0.142606, 0.287173], dtype=float32)

In [ ]:
mlx.nn.softmax(mlx.core.array([0.1, -0.2, 0.3, -0.2, 0.5])*8, axis=-1) # gets too peaky, converges to one-hot

array([0.0326083, 0.00295816, 0.16151, 0.00295816, 0.799965], dtype=float32)

In [ ]:
class LayerNorm1d: # (used to be BatchNorm1d)

  def __init__(self, dim, eps=1e-5, momentum=0.1):
    self.eps = eps
    self.gamma = mlx.core.ones(dim, dtype=mlx.core.float32)
    self.beta = mlx.core.zeros(dim, dtype=mlx.core.float32)

  def __call__(self, x):
    # calculate the forward pass
    xmean = x.mean(1, keepdims=True) # batch mean
    xvar = x.var(1, keepdims=True) # batch variance
    xhat = (x - xmean) / mlx.core.sqrt(xvar + self.eps) # normalize to unit variance
    self.out = self.gamma * xhat + self.beta
    return self.out

  def parameters(self):
    return [self.gamma, self.beta]

mlx.core.random.seed(1337)
module = LayerNorm1d(100)
x = mlx.core.random.normal((32, 100)) # batch size 32 of 100-dimensional vectors
x = module(x)
x.shape

(32, 100)

In [ ]:
x[:,0].mean(), x[:,0].std() # mean,std of one feature across all batch inputs

(array(-0.0968196, dtype=float32), array(0.963743, dtype=float32))

In [ ]:
x[0,:].mean(), x[0,:].std() # mean,std of a single input from the batch, of its features

(array(-1.19209e-08, dtype=float32), array(0.999994, dtype=float32))

In [ ]:
# French to English translation example:

# <--------- ENCODE ------------------><--------------- DECODE ----------------->
# les réseaux de neurones sont géniaux! <START> neural networks are awesome!<END>



### Full finished code, for reference

You may want to refer directly to the git repo instead though.

In [ ]:
import mlx.core
import mlx.nn
import mlx.optimizers

# hyperparameters
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 32 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 100
learning_rate = 1e-3
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.0
# ------------
# hyperparameters as in the video, would take about an hour to train on M4 Pro.
# batch_size = 64 # how many independent sequences will we process in parallel?
# block_size = 256 # what is the maximum context length for predictions?
# max_iters = 5000
# eval_interval = 500
# learning_rate = 3e-4
# eval_iters = 200
# n_embd = 384
# n_head = 6
# n_layer = 6
# dropout = 0.2
# ------------

mlx.core.random.seed(1337)

# wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = mlx.core.array(encode(text), dtype=mlx.core.int64)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = mlx.core.random.randint(0, len(data) - block_size, (batch_size,)).tolist()
    x = mlx.core.stack([data[i:i+block_size] for i in ix])
    y = mlx.core.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = mlx.core.zeros(eval_iters, dtype=mlx.core.float32)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Head(mlx.nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = mlx.nn.Linear(n_embd, head_size, bias=False)
        self.query = mlx.nn.Linear(n_embd, head_size, bias=False)
        self.value = mlx.nn.Linear(n_embd, head_size, bias=False)
        self._tril = mlx.core.tril(mlx.core.ones((block_size, block_size), dtype=mlx.core.float32))

        self.dropout = mlx.nn.Dropout(dropout)

    def __call__(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose((0, 2, 1)) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = mlx.core.where((self._tril[:T, :T] == 0), mlx.core.array(float('-inf')), wei) # (B, T, T)
        wei = mlx.nn.softmax(wei, axis=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(mlx.nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = [Head(head_size) for _ in range(num_heads)]
        self.proj = mlx.nn.Linear(n_embd, n_embd)
        self.dropout = mlx.nn.Dropout(dropout)

    def __call__(self, x):
        out = mlx.core.concatenate([h(x) for h in self.heads], axis=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(mlx.nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = mlx.nn.Sequential(
            mlx.nn.Linear(n_embd, 4 * n_embd),
            mlx.nn.ReLU(),
            mlx.nn.Linear(4 * n_embd, n_embd),
            mlx.nn.Dropout(dropout),
        )

    def __call__(self, x):
        return self.net(x)

class Block(mlx.nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = mlx.nn.LayerNorm(n_embd)
        self.ln2 = mlx.nn.LayerNorm(n_embd)

    def __call__(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

# super simple bigram model
class BigramLanguageModel(mlx.nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = mlx.nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = mlx.nn.Embedding(block_size, n_embd)
        self.blocks = mlx.nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = mlx.nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = mlx.nn.Linear(n_embd, vocab_size)

    def __call__(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(mlx.core.arange(T)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.reshape(B*T, C)
            targets = targets.reshape(B*T)
            loss = mlx.nn.losses.cross_entropy(logits, targets, reduction='mean')

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # sample from the distribution. pass unnormalized logits, dont softmax
            idx_next = mlx.core.random.categorical(logits, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = mlx.core.concatenate((idx, idx_next), axis=1) # (B, T+1)
            mlx.core.eval(idx) # force eval to release intermediate allocations
        return idx

model = BigramLanguageModel()
# put the model into training mode
model.train()
# print the number of parameters in the model
print(sum(p.size for _, p in mlx.utils.tree_flatten(model.trainable_parameters()))/1e6, 'M parameters')

# create an MLX optimizer
optimizer = mlx.optimizers.AdamW(learning_rate=learning_rate)

# Loss wrapper for value_and_grad
def loss_fn(module, idx, targets):
    _, loss = module(idx, targets)
    return loss

loss_and_grad_fn = mlx.nn.value_and_grad(model, loss_fn)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    loss, grads = loss_and_grad_fn(model, xb, yb)
    optimizer.update(model, grads)

    mlx.core.eval(model.parameters(), optimizer.state)

# put the model into evaluation mode
model.eval()
# generate from the model
context = mlx.core.zeros((1, 1), dtype=mlx.core.int64)
print(decode(model.generate(context, max_new_tokens=2000)[0].tolist()))


0.226113 M parameters
step 0: train loss 4.3217, val loss 4.3203
step 100: train loss 2.5267, val loss 2.5332
step 200: train loss 2.3858, val loss 2.3853
step 300: train loss 2.3034, val loss 2.3167
step 400: train loss 2.2353, val loss 2.2345
step 500: train loss 2.1554, val loss 2.1791
step 600: train loss 2.1049, val loss 2.1432
step 700: train loss 2.0667, val loss 2.1190
step 800: train loss 2.0401, val loss 2.0918
step 900: train loss 1.9901, val loss 2.0578
step 1000: train loss 1.9588, val loss 2.0441
step 1100: train loss 1.9421, val loss 2.0134
step 1200: train loss 1.9201, val loss 2.0177
step 1300: train loss 1.9002, val loss 2.0064
step 1400: train loss 1.8597, val loss 1.9810
step 1500: train loss 1.8478, val loss 1.9714
step 1600: train loss 1.8411, val loss 1.9674
step 1700: train loss 1.8220, val loss 1.9549
step 1800: train loss 1.8125, val loss 1.9342
step 1900: train loss 1.7977, val loss 1.9340
step 2000: train loss 1.7894, val loss 1.9153
step 2100: train loss 1.